# JAGUAR E-TYPE DATA PULL

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import joblib
import os
import sys
import helpers

Get List of Every Result and Save:

In [121]:
#### Get List of Every Result and Save ####
#### DO NOT DELETE ####

make = input('Enter Car Make: ')
model = input('Enter Car Model: ')
open("./data/HTML/"+ make + model +".html", 'w').close()
ids, urls = helpers.getlistings(make, model)
data = ''
listings = []
for i in range(0,len(ids)):
    urlstart = 'https://bringatrailer.com/wp-json/bringatrailer/1.0/data/keyword-filter?bat_keyword_pages=' + ids[i] + '&sort=td&page='
    urlend = '&results=items'
    pgend = 15
    tempurl = helpers.geturls(urlstart, urlend, pgend, urls)
    listings = tempurl
    listings = listings[len(ids):len(listings)]
    def get_data(url):

        r = requests.get(url, allow_redirects=False)

        soup = BeautifulSoup(r.content, "html.parser")
        return str(soup)


    souplist = joblib.Parallel(n_jobs=-2)(joblib.delayed(get_data)(url) for url in listings)


### WRITE DATA TO FILE ###
    delimiter = "%newline%"
    temp = delimiter.join(souplist)
    data = temp + delimiter + data
with open("./data/HTML/"+ make + model +".html", "a") as file:
    file.write(data)


display('done')

'done'

Pull Information Off Each Listing Stored in URLS List:

In [122]:
carlist = []
Car = []
### READ DATA ###

with open("./data/HTML/"+ make + model +".html", "r") as file:
    test_data = file.read()

### Write Data to DF ###
out = test_data.split(delimiter)

for i in range(0,len(listings)):
    soup = BeautifulSoup(out[i], 'html.parser')
    title = str(soup.find('title'))
    if "Parts" not in title and "Tool" not in title and "Memorabilia" not in title and "Luggage" not in title and "Tool Kit" not in title and "Removable" not in title and "Gearbox" not in title and title != 'None' and 'listing' in listings[i]:
        loc = str(soup.findAll('a'))
        transmission = str(soup.findAll('li'))
        contents = str(soup.findAll('p'))
        essentials = soup.find('div', class_='essentials')

        #Get Car Description
        desc = helpers.getdesc(listings[i])


        #Get Year
        year = desc[0:4]
        
        #Check if Sold and get highest bid
        sold, price = helpers.getsaleprice(title, loc)


        #Get Transmission
        trans = 'Manual'
        if 'Automatic' in transmission:
            trans = 'Automatic'

        #Get Location

        town, state = helpers.getlocation(loc)

        
        #Check for Project
        if 'project' in title or 'Project' in title:
            project = 1
        else:
            project = 0
        
        #Get Lot Number
        try:
            index = title.index('Lot #')
            lotnumb = title[index+5:index+12].partition(')')[0]
        except:
            lotnumb = "N/A"

        #Check for Replica
        if "replica" in title or "Replica" in title:
            replica = 1
        else:
            replica = 0

        #Get Month
        month = helpers.getmonth(title)
        
        #Get Year
        if '(' in title:
            index = title.index('(')
            yearsold = title[index-5:index-1]
        else:
            index = title.index('20')
            yearsold = title[index:index+5]
        
        #Get Engine and Number Matching
        numbmatch = 0
        
        if 'Numbers-Matching' in transmission:
            numbmatch = 1

        engine = helpers.getengine(transmission)
        
        #Get Various Indicators for rust, refurbishment, restoration, scratch, paint bubbles, metal repair, hardtop, or overdrive options
        rust, refurbished, restored, scratch, paintbub, metalrepair, hardtop, overdrive, turbo, super = helpers.getindicators(contents)
            
            
        #Get Mileage
        miles, milestmu = helpers.getmileage(transmission)

        #Get Descriptions of options and chassis #

        temp = helpers.getenginedesc(essentials, engine)
        chassis = temp[0]
        specialdesc = temp[1]
        mileagedesc = temp[2]
        enginedesc = temp[3]
        transdesc = temp[4]
        paintdesc = temp[5]
        interiordesc = temp[6]
        carbdesc = temp[7]
        wheeldesc = temp[8]
        brakedesc = temp[9]
        suspdesc = temp[10]
        engine = temp[11]

        #Check Trans with Description
        if 'Manual' in transdesc:
            trans = 'Manual'
        elif 'Automatic' in transdesc:
            trans = 'Automatic'

        #Check Mileage with Description
        try:
            if "K" in str(miles):
                index = mileagedesc.index("ilometers")
                tempmile = mileagedesc[0:index].partition(" ")[0]
                if "k" in tempmile or "K" in tempmile:
                    milestmu = re.sub(r"[^0-9]", "", tempmile)
                    milestmu = int(milestmu) * 1000
            elif "K" in str(milestmu):
                index = mileagedesc.index("ilometers")
                tempmile = mileagedesc[0:index].partition(" ")[0]
                if "k" in tempmile or "K" in tempmile:
                    miles = re.sub(r"[^0-9]", "", tempmile)
                    miles = int(miles) * 1000
        except:
            print('failed')
        #Check for bespoke bodies
        if 'Bertone' in title:
            bertone = 1
        else:
            bertone = 0
        if 'Zagato' in title:
            zagato = 1
        else:
            zagato = 0

        Car = [lotnumb, desc, trans, year, miles, milestmu, engine, turbo, super, numbmatch, hardtop, overdrive, scratch, paintbub, metalrepair, replica, rust, refurbished, restored, project, sold, month, yearsold, price, town, state, zagato, bertone, chassis, specialdesc, mileagedesc, enginedesc, transdesc, paintdesc, interiordesc, carbdesc, wheeldesc, brakedesc, suspdesc, listings[i], make, model]
        carlist.append(Car)
    else:
        print('Discarded')
    # index = transmission.index('aint')
    # print(transmission[index-20:index+20])

df = pd.DataFrame(carlist, columns=['Lot Number', 'Description', 'Transmission', 'Model Year', 'Mileage', 'Mileage (if TMU)','Engine', 'Turbocharged', 'Supercharged', 'Numbers Matching', 'Hardtop', 'Overdrive', 'Scratch', 'Paint Bubble', 'Metal Repair', 'Replica', 'Rust','Refurbished','Restored', 'Project', 'Sold', 'Month', 'Year', 'Price', 'City', 'State', 'Zagato', 'Bertone', 'Chassis', 'Special Description', 'Mileage Description', 'Engine Description', 'Transmission Description','Paint Description', 'Interior Description','Carburetor Description', 'Wheels Description', 'Brakes Description', 'Suspension Description', 'URL', 'Make', 'Model'])

display(df)



,Lot Number,Description,Transmission,Model Year,Mileage,Mileage (if TMU),Engine,Turbocharged,Supercharged,Numbers Matching,...,Transmission Description,Paint Description,Interior Description,Carburetor Description,Wheels Description,Brakes Description,Suspension Description,URL,Make,Model
0,"93,984",1971 nissan skyline h t 2000gt r hakosuka,Manual,1971,59000,N/A - K,2.0,0,0,0,...,Five-Speed Manual Transmission,Silver Paint,Black Vinyl Upholstery,Triple Mikuni Solex Carburetors,Staggered SSR Basketweave Wheels,Front Disc Brakes,Four-Wheel Independent Suspension,https://bringatrailer.com/listing/1971-nissan-...,Nissan,C
1,"92,321",1972 nissan skyline 2000 gt r,Manual,1972,,N/A,2.0,0,0,0,...,Five-Speed Manual Transmission,White Paint w/Red Livery,Black Upholstery,N/A,"16"" Panasport Alloy Wheels",Four-Wheel Disc Brakes,Coilover Suspension,https://bringatrailer.com/listing/1972-nissan-...,Nissan,C
2,"91,686",1969 nissan skyline 2000 gt r,Manual,1969,TMU - K,84000,2.0,0,0,0,...,Five-Speed Manual Transmission,White Paint,Black Upholstery,N/A,"14"" Watanabe Wheels",Front Disc Brakes,N/A,https://bringatrailer.com/listing/1969-nissan-...,Nissan,C
3,"85,354",1972 nissan skyline 2000 gt gc10,Manual,1972,TMU - K,93000,2.8,0,0,0,...,Five-Speed Manual Transmission,White Paint,Black Vinyl Upholstery,Triple Weber Carburetors,"14"" Watanabe Wheels",Front Disc Brakes,N/A,https://bringatrailer.com/listing/1972-nissan-...,Nissan,C
4,"74,113",1969 nissan skyline 2000 gt r,Manual,1969,TMU - K,84000,2.0,0,0,0,...,5-Speed Manual Transmission,White Paint,Black Upholstery,N/A,"14"" Watanabe Wheels",Front Disc Brakes,N/A,https://bringatrailer.com/listing/1969-nissan-...,Nissan,C
5,"63,600",1971 nissan skyline kgc10,Manual,1971,TMU - K,42000,3.0,0,0,0,...,Five-Speed Manual Transmission,Repainted Silver,Black Vinyl Upholstery,Triple Mikuni Carburetors,"15"" Watanabe-Style Wheels",N/A,N/A,https://bringatrailer.com/listing/1971-nissan-...,Nissan,C
6,"53,634",1971 nissan skyline c10,Manual,1971,TMU,700,2.5,0,0,0,...,5-Speed RB25DE Manual Transmission,White Paint,Black Vinyl Interior,N/A,"16"" American Racing Alloy Wheels",4-Wheel Disc Brakes,N/A,https://bringatrailer.com/listing/1971-nissan-...,Nissan,C
7,"42,806",1972 nissan skyline 2000gt gt r tribute,Manual,1972,TMU,42,3.2,0,0,0,...,Five-Speed Manual Gearbox,N/A,Black Vinyl Upholstery,N/A,"16"" Watanabe Wheels",Four-Wheel Disc Brakes,N/A,https://bringatrailer.com/listing/1972-nissan-...,Nissan,C
8,"35,576",1972 nissan skyline c10,Manual,1972,93000,N/A - K,2.8,0,0,0,...,Five-Speed Manual Transmission,N/A,N/A,Triple Mikuni Carburetors,"15"" SSR Longchamp Wheels",N/A,Coilover Suspension,https://bringatrailer.com/listing/1972-nissan-...,Nissan,C
9,"24,917",1972 nissan skyline 2000 gt gc10,Manual,1972,22000,N/A - K,1.5,0,0,0,...,Four-Speed Manual Transmission,N/A,N/A,N/A,"13"" Alloy Wheels",N/A,N/A,https://bringatrailer.com/listing/1972-nissan-...,Nissan,C


Save Data as Excel File:

In [123]:
path = 'data/' + make + '/'
try:
    os.mkdir(path)
except:
    print('Path already exists')
df.to_excel(path + make + '_' + model + '.xlsx')
print('Saved!')

Path already exists
Saved!
